In [1]:
# import the libraries
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm
from datetime import datetime, timedelta
import random

In [2]:
# fetch stock data from yahoo finance
def fetch_stock_data(ticker, start_date, end_date):
  stock_data = yf.download(ticker, start=start_date, end=end_date)
  return stock_data

# we will use Apple stock for this use-case
ticker = 'AAPL'
stock_data = fetch_stock_data(ticker, '2019-01-01', '2021-01-01')

# display first few rows of the stock data
stock_data.head()

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2019-01-02,37.708595,39.480000,39.712502,38.557499,38.722500,148158800
2019-01-03,33.952541,35.547501,36.430000,35.500000,35.994999,365248800
2019-01-04,35.401962,37.064999,37.137501,35.950001,36.132500,234428400
2019-01-07,35.323158,36.982498,37.207500,36.474998,37.174999,219111200
2019-01-08,35.996529,37.687500,37.955002,37.130001,37.389999,164101200


In [3]:
# calculate historical volatility
def calculate_volatility(stock_data, window=30):
  stock_data['Daily Return'] = stock_data['Adj Close'].pct_change()
  stock_data['Volatility'] = stock_data['Daily Return'].rolling(window=window).std() * np.sqrt(252)
  return stock_data

# calculate volatility
stock_data = calculate_volatility(stock_data)

# display last few rows to check volatility
stock_data[['Adj Close', 'Volatility']].tail()

Price,Adj Close,Volatility
Ticker,AAPL,
Date,,
2020-12-24,129.047501,0.251968
2020-12-28,133.662994,0.268064
2020-12-29,131.883255,0.272886
2020-12-30,130.758743,0.275001
2020-12-31,129.751602,0.275050


In [4]:
# Implement the Black-Scholes Model for Call/Put Options
def black_scholes(S, K, T, r, sigma, option_type='call'):
  d1 = (np.log(S/K) + (r + (sigma**2)/2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)

  if option_type == 'call':
    option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
  elif option_type == 'put':
    option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

  return option_price

In [5]:
# Generate synthetic option data
def generate_option_data(stock_data, strike_range=0.05, days_to_expiration=30):
    option_data = []
    current_date = stock_data.index[-1]  # Last date in stock data

    # Ensure 'Adj Close' column is valid
    if 'Adj Close' not in stock_data.columns:
        raise ValueError("Column 'Adj Close' not found in the stock_data DataFrame")

    # Convert 'Adj Close' to a valid series
    stock_prices = stock_data['Adj Close'].squeeze()

    for stock_price in stock_prices.dropna().iloc[-5:]:  # Use the last 5 valid stock prices
        strike_price = round(stock_price * (1 + random.uniform(-strike_range, strike_range)), 2)
        expiration_date = current_date + timedelta(days=days_to_expiration)  # Always set to a future date
        option_data.append({
            'Date': current_date,
            'Strike Price': strike_price,
            'Stock Price': stock_price,
            'Expiration Date': expiration_date,
            'Option Type': 'call' if random.random() > 0.5 else 'put',  # Randomly choose call or put
            'Option Price': round(random.uniform(1, 10), 2),  # Random option price
            'Implied Volatility': round(random.uniform(0.2, 0.4), 2)  # Random implied volatility
        })

    return pd.DataFrame(option_data)

# Regenerate synthetic option data with corrected expiration dates
option_data = generate_option_data(stock_data)

In [6]:
# Add Time to Maturity (in years) to the option data
option_data['Time to Maturity'] = (option_data['Expiration Date'] - option_data['Date']).dt.days / 365

# Check for invalid or negative values
print(option_data[['Expiration Date', 'Time to Maturity']].head())

  Expiration Date  Time to Maturity
0      2021-01-30          0.082192
1      2021-01-30          0.082192
2      2021-01-30          0.082192
3      2021-01-30          0.082192
4      2021-01-30          0.082192


In [7]:
# compare theorteical prices with market prices
risk_free_rate = 0.02 # 2% annual risk-free rate

def compare_prices(option_data, stock_data, risk_free_rate):
  theoretical_prices = []
  for idx, row in option_data.iterrows():
    S = row['Stock Price']
    K = row['Strike Price']
    T = row['Time to Maturity']
    sigma = row['Implied Volatility'] # use implied volatility for this comparison
    option_type = row['Option Type']

    # calculate theoretical option price using Black-Scholes Model
    theoretical_price = black_scholes(S, K, T, risk_free_rate, sigma, option_type)
    theoretical_prices.append(theoretical_price)

  option_data['Theoretical Price'] = theoretical_price

  # calculate errors (absolute difference b/w real market price and theoretical price)
  option_data['Price Error'] = abs(option_data['Option Price'] - option_data['Theoretical Price'])

  return option_data

# compare prices and calculate error metrics
option_data = compare_prices(option_data, stock_data, risk_free_rate)

# display the results
option_data[['Date', 'Option Type', 'Strike Price', 'Stock Price', 'Expiration Date', 'Option Price', 'Implied Volatility', 'Theoretical Price', 'Price Error']].head()

,Date,Option Type,Strike Price,Stock Price,Expiration Date,Option Price,Implied Volatility,Theoretical Price,Price Error
0,2020-12-31,put,133.19,129.047501,2021-01-30,5.10,0.27,6.799675,1.699675
1,2020-12-31,put,136.97,133.662994,2021-01-30,8.77,0.21,6.799675,1.970325
2,2020-12-31,call,130.60,131.883255,2021-01-30,5.41,0.36,6.799675,1.389675
3,2020-12-31,call,128.70,130.758743,2021-01-30,8.41,0.39,6.799675,1.610325
4,2020-12-31,call,125.54,129.751602,2021-01-30,1.66,0.29,6.799675,5.139675


In [8]:
# evaluation metrics
def evaluate_metrics(option_data):
  # mean absolute error (MAE)
  mae = option_data['Price Error'].mean()

  # Root-Mean Squared Error (RMSE)
  rmse = np.sqrt((option_data['Price Error'] ** 2).mean())

  # Mean Error (Bias)
  mean_error = option_data['Price Error'].mean()

  print(f"Mean Absolute Error (MAE): {mae}")
  print(f"Root Mean Squared Error (RMSE): {rmse}")
  print(f"Mean Error (Bias): {mean_error}")

# evaluate metrics
evaluate_metrics(option_data)

Mean Absolute Error (MAE): 2.3619349861072396
Root Mean Squared Error (RMSE): 2.746331275805504
Mean Error (Bias): 2.3619349861072396
